In [1]:
import pandas as pd
import numpy as np
import datetime
from functools import reduce

data = pd.read_csv("sars_cases.csv")
death = pd.read_csv("sars_deaths.csv")
hospitalization = pd.read_csv("sars_hospitalizations.csv")
#adding suffix
hospitalization = hospitalization.add_suffix("_hospitalization")
census = pd.read_csv("Census_Wash_Counties_Income_Poverty.csv")

cleandata = pd.DataFrame.copy(data)

day = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[1]
month = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[0]
year = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[2]

month = month.map(lambda x: x.zfill(2))
day = day.map(lambda x: x.zfill(2))


cleandata["WeekStartDate"] = year + "-" + month + "-" + day




dataframes = [cleandata, death, hospitalization]

#mergedf = reduce(lambda  left,right: pd.merge(left,right,on=['County', "WeekStartDate"], suffixes= ("_cases", "_death", "_hospitalization"), how='outer'), dataframes)

mergedf = pd.merge(cleandata, death, on = ["County", "WeekStartDate"], suffixes = ("_cases", "_death"), how= "outer", indicator = True)#.fillna(0)
#mergedf[mergedf["_merge"] == "both"]

merge1 = pd.DataFrame(mergedf)

mergedf2 = pd.merge(merge1, hospitalization, left_on = ["County", "WeekStartDate"], right_on = ["County_hospitalization", "WeekStartDate_hospitalization"], how = "outer", indicator = "exists")

#mergedf.fillna(0)

mergedf2.to_csv("test.csv")
mergedf2

,County,WeekStartDate,NewPos_All,Age 0-19_cases,Age 20-39_cases,Age 40-59_cases,Age 60-79_cases,Age 80+_cases,Positive UnkAge_cases,dtm_updated_cases,...,WeekStartDate_hospitalization,Hospitalizations_hospitalization,Age 0-19_hospitalization,Age 20-39_hospitalization,Age 40-59_hospitalization,Age 60-79_hospitalization,Age 80+_hospitalization,Positive UnkAge_hospitalization,dtm_updated_hospitalization,exists
0,Adams County,2020-03-08,4.0,0.0,1.0,1.0,2.0,0.0,0.0,7/5/2020 15:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Adams County,2020-03-15,3.0,0.0,0.0,3.0,0.0,0.0,0.0,7/5/2020 15:36,...,2020-03-15,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2020/07/05 15:36:42,both
2,Adams County,2020-03-22,9.0,0.0,1.0,4.0,4.0,0.0,0.0,7/5/2020 15:36,...,2020-03-22,4.0,0.0,0.0,1.0,3.0,0.0,0.0,2020/07/05 15:36:42,both
3,Adams County,2020-03-29,17.0,2.0,8.0,4.0,3.0,0.0,0.0,7/5/2020 15:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Adams County,2020-04-05,8.0,1.0,2.0,4.0,1.0,0.0,0.0,7/5/2020 15:36,...,2020-04-05,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2020/07/05 15:36:42,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,6.0,1.0,2.0,2.0,1.0,0.0,0.0,2020/07/05 15:36:42,right_only
610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,3.0,0.0,0.0,1.0,1.0,1.0,0.0,2020/07/05 15:36:42,right_only
611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2020/07/05 15:36:42,right_only
612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2020/07/05 15:36:42,right_only


In [2]:
mergedf3 = mergedf2
mergedf3["Month"] = mergedf3["WeekStartDate"].str.split("-").str[1]

mergemonth = mergedf3.groupby(["County", "Month"])
mergemonth = mergemonth[['NewPos_All', 'Age 0-19_cases',
       'Age 20-39_cases', 'Age 40-59_cases', 'Age 60-79_cases',
       'Age 80+_cases', 'Positive UnkAge_cases', 'dtm_updated_cases', 'Deaths',
       'Age 0-19_death', 'Age 20-39_death', 'Age 40-59_death',
       'Age 60-79_death', 'Age 80+_death', 'Positive UnkAge_death',
       'dtm_updated_death', 'County_hospitalization',
       'Hospitalizations_hospitalization',
       'Age 0-19_hospitalization', 'Age 20-39_hospitalization',
       'Age 40-59_hospitalization', 'Age 60-79_hospitalization',
       'Age 80+_hospitalization', 'Positive UnkAge_hospitalization',
       'dtm_updated_hospitalization']].sum()

mergemonth

NewPos_All  Age 0-19_cases  Age 20-39_cases  \
County        Month                                                
Adams County  03           33.0             2.0             10.0   
              04           16.0             1.0              8.0   
              05           40.0             9.0             17.0   
              06           77.0             5.0             28.0   
Asotin County 03            4.0             0.0              0.0   
...                         ...             ...              ...   
Yakima County 02            5.0             0.0              1.0   
              03          501.0            22.0            144.0   
              04         1175.0           103.0            412.0   
              05         3528.0           441.0           1515.0   
              06         2279.0           362.0            952.0   

                     Age 40-59_cases  Age 60-79_cases  Age 80+_cases  \
County        Month                                                    
Adams County  03                12.0              9.0            0.0   
              04                 6.0              1.0            0.0   
              05                12.0              2.0            0.0   
              06                29.0             13.0            0.0   
Asotin County 03                 2.0              2.0            0.0   
...                              ...              ...            ...   
Yakima County 02                 3.0              1.0            0.0   
              03               163.0            125.0           46.0   
              04               392.0            190.0           77.0   
              05              1085.0            407.0           80.0   
              06               593.0            248.0          124.0   

                     Positive UnkAge_cases  Deaths  Age 0-19_death  \
County        Month                                                  
Adams County  03                       0.0     0.0             0.0   
              04                       0.0     0.0             0.0   
              05                       0.0     0.0             0.0   
              06                       2.0     0.0             0.0   
Asotin County 03                       0.0     1.0             0.0   
...                                    ...     ...             ...   
Yakima County 02                       0.0     0.0             0.0   
              03                       1.0    35.0             0.0   
              04                       1.0    52.0             0.0   
              05                       0.0    46.0             0.0   
              06                       0.0    27.0             0.0   

                     Age 20-39_death  ...  Age 60-79_death  Age 80+_death  \
County        Month                   ...                                   
Adams County  03                 0.0  ...              0.0            0.0   
              04                 0.0  ...              0.0            0.0   
              05                 0.0  ...              0.0            0.0   
              06                 0.0  ...              0.0            0.0   
Asotin County 03                 0.0  ...              1.0            0.0   
...                              ...  ...              ...            ...   
Yakima County 02                 0.0  ...              0.0            0.0   
              03                 1.0  ...             13.0           18.0   
              04                 1.0  ...             24.0           22.0   
              05                 1.0  ...             21.0           15.0   
              06                 0.0  ...              8.0           14.0   

                     Positive UnkAge_death  Hospitalizations_hospitalization  \
County        Month                                                            
Adams County  03                       0.0                               5.0   
              04                       0.0    

In [3]:
data.head()

,County,WeekStartDate,NewPos_All,Age 0-19,Age 20-39,Age 40-59,Age 60-79,Age 80+,Positive UnkAge,dtm_updated
0,Adams County,3/8/2020,4,0,1,1,2,0,0,7/5/2020 15:36
1,Adams County,3/15/2020,3,0,0,3,0,0,0,7/5/2020 15:36
2,Adams County,3/22/2020,9,0,1,4,4,0,0,7/5/2020 15:36
3,Adams County,3/29/2020,17,2,8,4,3,0,0,7/5/2020 15:36
4,Adams County,4/5/2020,8,1,2,4,1,0,0,7/5/2020 15:36


In [4]:
death.head()

,County,WeekStartDate,Deaths,Age 0-19,Age 20-39,Age 40-59,Age 60-79,Age 80+,Positive UnkAge,dtm_updated
0,Asotin County,2020-03-15,1,0,0,0,1,0,0,2020/07/05 15:36:42
1,Asotin County,2020-04-05,1,0,0,0,0,1,0,2020/07/05 15:36:42
2,Benton County,2020-03-01,3,0,0,0,1,2,0,2020/07/05 15:36:42
3,Benton County,2020-03-08,4,0,0,0,3,1,0,2020/07/05 15:36:42
4,Benton County,2020-03-15,8,0,0,0,5,3,0,2020/07/05 15:36:42


In [5]:
hospitalization.head()

,County_hospitalization,WeekStartDate_hospitalization,Hospitalizations_hospitalization,Age 0-19_hospitalization,Age 20-39_hospitalization,Age 40-59_hospitalization,Age 60-79_hospitalization,Age 80+_hospitalization,Positive UnkAge_hospitalization,dtm_updated_hospitalization
0,Adams County,2020-03-15,1,0,0,1,0,0,0,2020/07/05 15:36:42
1,Adams County,2020-03-22,4,0,0,1,3,0,0,2020/07/05 15:36:42
2,Adams County,2020-04-05,1,0,0,1,0,0,0,2020/07/05 15:36:42
3,Adams County,2020-06-07,1,0,0,1,0,0,0,2020/07/05 15:36:42
4,Adams County,2020-06-14,1,0,0,1,0,0,0,2020/07/05 15:36:42


In [6]:
census.head()

,GEO_ID,NAME,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,...,DP03_0135PE,DP03_0135PM,DP03_0136E,DP03_0136M,DP03_0136PE,DP03_0136PM,DP03_0137E,DP03_0137M,DP03_0137PE,DP03_0137PM
0,id,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Percent Estimate!!EMPLOYMENT STATUS!!Populatio...,Percent Margin of Error!!EMPLOYMENT STATUS!!Po...,...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Margin of Error!!PERCENTAGE OF FAMILIES AND PE...,Percent Estimate!!PERCENTAGE OF FAMILIES AND P...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
1,0500000US53011,"Clark County, Washington",380248,1154,380248,(X),247061,3303,65,0.9,...,6.7,1.8,(X),(X),5.9,1.2,(X),(X),21.3,2.2
2,0500000US53015,"Cowlitz County, Washington",86221,563,86221,(X),50611,2397,58.7,2.7,...,5.5,2.5,(X),(X),8.6,2.7,(X),(X),20.8,4.5
3,0500000US53041,"Lewis County, Washington",64562,576,64562,(X),34306,1531,53.1,2.4,...,7,2.4,(X),(X),7.4,2.7,(X),(X),25.5,5.8
4,0500000US53045,"Mason County, Washington",54199,413,54199,(X),28311,2018,52.2,3.8,...,6.6,3.4,(X),(X),11.8,3.9,(X),(X),19.3,6.1
